<a href="https://colab.research.google.com/github/afontoura/DRLA2ALM/blob/master/stable_baselines_DDPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable_baselines==2.9.0
!git clone https://github.com/afontoura/DRLA2ALM
%cd DRLA2ALM
!pip install -e .

     |████████████████████████████████| 235kB 3.4MB/s 
  Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1
Cloning into 'DRLA2ALM'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 211 (delta 31), reused 159 (delta 17), pack-reused 0
Receiving objects: 100% (211/211), 22.26 MiB | 49.87 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/DRLA2ALM
Obtaining file:///content/DRLA2ALM
  Running setup.py develop for DRLA2ALM


In [2]:
import gym
import envs
import numpy as np

from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG

time_horizon = 80
discount_rate = .08
current_seed = 10

env = gym.make('ALM-v0', T = time_horizon, rate = discount_rate)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
# the noise objects for DDPG
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

model = DDPG(MlpPolicy, env, verbose=1, param_noise=param_noise, action_noise=action_noise, seed = current_seed)
model.learn(total_timesteps=500000)






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




--------------------------------------
| reference_Q_mean        | 4.36     |
| reference_Q_std         | 0.406    |
| reference_action_mean   | -0.529   |
| reference_action_std    | 0.755    |
| reference_actor_Q_mean  | 4.36     |
| reference_actor_Q_std   | 0.406    |
| rollout/Q_mean          | 2.42     |
| rollout/actions_mean    | -0.267   |
| rollout/actions_std     | 0.628    |
| rollout/episode_steps   | 54.3     |
| rollout/episodes        | 184      |
| rollout/return          | 53.9     |
| rollout/return_history  | 53.7     |
| total/duration          | 43.4     |
| total/episodes          | 184      |
| total/epochs            | 1        |
| total/steps             | 9998     |


In [0]:
resultados = []
for i in np.arange(10):
  dones = False
  G = []
  obs = env.reset()
  while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    G.append(rewards)
  resultados.append(np.array(G).sum())
  #print(str(i) + ': ' + str(np.array(G).sum()))

In [15]:
print((np.array(resultados) == time_horizon).sum())
print(np.array(resultados).mean())

5
51.2


In [0]:
model.save('ddpg_t' + str(time_horizon) + '_dr' + str(round(discount_rate * 1000)) + '_seed' + str(current_seed))